In [ ]:
import pandas as pd
from tools.trainer import TrainerCon
from models.SinusoidalModel import SinSiameseModel
import torch
from torch.utils.data import DataLoader
import numpy as np
from sklearn.preprocessing import LabelEncoder
import tools.utils as uti
from tools.con_tran_data import ConTranDataGen
from tqdm import tqdm
from collections import defaultdict
import sys

In [ ]:
import seaborn as sns
import datetime
from sklearn.metrics import accuracy_score
import matplotlib.pyplot as plt
from sklearn.cluster import *

import swifter

from scipy.sparse import csr_matrix

In [ ]:
import pickle
from rdkit import Chem
import pickle
from tqdm.notebook import tqdm
import numpy as np
import pandas as pd
from scipy.spatial.distance import cdist

from models.SinusoidalModel import SinSiameseModel
import torch
from matchms.filtering.add_fingerprint import add_fingerprint
from matchms.similarity import FingerprintSimilarity
import random
from sklearn.preprocessing import LabelEncoder
import os
from sklearn.preprocessing import LabelEncoder
from sklearn.manifold import TSNE
import matplotlib.pyplot as plt
import collections

import joblib
import numpy as np
from sklearn import metrics
import time

plt.rc('font',family='Times New Roman')

# init parameters

In [ ]:
abspath = "/data1/CLERMAS/"
score_name = 'daylight' 
score_df = pd.read_pickle(abspath + f"data/ALL_GNPS_210125_positive_{score_name}_tanimoto_scores.pkl") # 相似分数
pth = abspath + "dict/" # model save path
val_ratio = 0.2 # validation 
sim_loss =  30 # structual similarity loss weight
epochs = 8
project_size = 200 # the embedding size
temperature = 0.05 # the temperature
model_name = f"pre_T{temperature}_sim_loss{sim_loss}_epochs{epochs}"
PRECURSOR_MZ = "precursor_mz_new"
train_flag = 1 # means traing, if 0 means just predict

In [ ]:
model_results = pth + model_name
if not os.path.exists(model_results):
    os.mkdir(model_results)
    os.mkdir(model_results + "/models")

# function

In [ ]:
def get_label(x):
    return x

In [ ]:
def true_M(precursor_mz, adduct):
    if adduct == "0" or adduct=="[Cat]" or adduct=="[M+HFA+H]+" or adduct=="[M+HFA+NH4]+":
        return precursor_mz
    elif adduct == "[2M+Ca-H]+":
        return (precursor_mz-39)/2
    elif adduct == "[2M+Ca]2+":
        return precursor_mz-20
    elif adduct == "[2M+H+CH3CN]":
        return (precursor_mz-42 )/2
    elif adduct == "[2M+H]":
        return (precursor_mz-1)/2
    elif adduct =="[2M+H]+":
        return (precursor_mz-1)/2
    elif adduct =="[2M+K]+":
        return (precursor_mz-39)/2
    elif adduct =="[2M+NH4]":
        return (precursor_mz-16)/2
    elif adduct =="[2M+NH4]+":
        return (precursor_mz-16)/2
    elif adduct =="[2M+Na]+":
        return (precursor_mz-23)/2
    elif adduct=="[2M-2H2O+H]+":
        return (precursor_mz+35)/2
    elif adduct=="[2M-H+2Na]+":
        return (precursor_mz-45)/2
    elif adduct=="[2M-H2O+H]+":
        return (precursor_mz+17)/2
    elif adduct=="[2M-H2O+Na]+":
        return (precursor_mz-5)/2
    elif adduct=="[3M+Ca-H]+":
        return (precursor_mz-39)/3
    elif adduct=="[3M+Ca]2+":
        return (precursor_mz-40)/3
    elif adduct=="[3M+H]" or adduct=="[3M+H]+":
        return (precursor_mz-1)/3
    elif adduct=="[3M+K]+":
        return (precursor_mz-39)/3
    elif adduct=="[3M+NH4]" or adduct=="[3M+NH4]+":
        return (precursor_mz-18)/3
    elif adduct=="[3M+Na]" or adduct=="[3M+Na]+":
        return (precursor_mz-23)/3
    elif adduct=="[4M+Ca]2+":
        return (precursor_mz*2-40)/4
    elif adduct=="[M+15]+":
        return precursor_mz-15
    elif adduct=="[M+2H+2]" or adduct=="[M+2H]2+":
        return precursor_mz*2 -2
    elif adduct=="[M+2Na-H]+":
        return precursor_mz - 45
    elif adduct=="[M+2Na]":
        return precursor_mz - 46
    elif adduct=="[M+3H]3+":
        return precursor_mz*3 -3
    elif adduct=="[M+ACN+NH4]+":
        return precursor_mz - 59
    elif adduct=="[M+ACN+H]+":
        return precursor_mz - 42
    elif adduct=="[M+CH3OH+H]":
        return precursor_mz-33
    elif adduct=="[M+Ca-H]+":
        return precursor_mz-39
    elif adduct=="[M+Ca]2+":
        return precursor_mz*2-40
    elif adduct=="[M+Cl]-":
        return precursor_mz-35.5
    elif adduct=="[M+H+C2H6OS]":
        return precursor_mz-79
    elif adduct=="[M+H+CH3CN]":
        return precursor_mz-42
    elif adduct=="[M+H+CH3OH]":
        return precursor_mz-37
    elif adduct=="[M+H+HCOOH]":
        return precursor_mz-47
    elif adduct=="[M+H+Na]2+":
        return 2*precursor_mz-24
    elif adduct=="[M+H-(C12H20O9)]+":
        return precursor_mz+307
    elif adduct=="[M+H-2H2O]+":
        return precursor_mz + 35
    elif adduct=="[M+H-99]":
        return precursor_mz+98
    elif adduct=="[M+H-C9H10O5]":
        return precursor_mz +197
    elif adduct=="[M+H-H20]":
        return precursor_mz+17
    elif adduct=="[M+H-NH3]":
        return precursor_mz+14
    elif adduct=="[M+HCl]":
        return precursor_mz-36.5
    elif adduct=="[M+H]+":
        return precursor_mz-1
    elif adduct=="[M+K]+":
        return precursor_mz-39
    elif adduct=="[M+Li]+":
        return precursor_mz-7
    elif adduct=="[M+NH3]":
        return precursor_mz-15
    elif adduct=="[M+NH4-H2O]":
        return precursor_mz+2
    elif adduct=="[M+NH4]+":
        return precursor_mz-16
    elif adduct=="[M+Na+CH3CN]":
        return precursor_mz-64
    elif adduct=="[M+Na]+":
        return precursor_mz-23
    elif adduct=="[M-2(H2O)+H]" or adduct=="[M-2H2O+H]+":
        return precursor_mz+35
    elif adduct=="[M-2H2O+2H]2+":
        return precursor_mz+34
    elif adduct=="[M-2H2O+NH4]+":
        return precursor_mz+18
    elif adduct=="[M-3H2O+2H]2+":
        return precursor_mz*2-2+3*18
    elif adduct=="[M-3H2O+H]" or adduct=="[M-3H2O+H]+":
        return precursor_mz+3*18-1
    elif adduct=="[M-4H2O+H]+":
        return precursor_mz+4*18-1
    elif adduct=="[M-5H2O+H]+":
        return precursor_mz+5*18-1
    elif adduct=="[M-C6H10O5+H]":
        return precursor_mz+12*6+10+16*5-1
    elif adduct=="[M-H+Li]+":
        return precursor_mz-6
    elif adduct=="[M-H+Na]+":
        return precursor_mz-22
    elif adduct=="[M-H2O+H]+":
        return precursor_mz+17
    elif adduct=="[M-H2O+NH4]+":
        return precursor_mz
    elif adduct=="[M-H2O]":
        return precursor_mz + 18
    elif adduct=="[M-H]-":
        return precursor_mz+1
    elif adduct=="[M-MeOH+H]":
        return precursor_mz
    elif adduct=="[M2+H]":
        return (precursor_mz-1)/2
    elif adduct=="[M2+Na]":
        return (precursor_mz-23)/2
    elif adduct=="[M2Br81+H]":
        return (precursor_mz-1)/2
    elif adduct=="[MBr81+H]":
        return precursor_mz-1
    elif adduct=="[MCl+H]":
        return precursor_mz -1
    elif adduct=="[MCl37+H]":
        return precursor_mz-1
    elif adduct=="[MS+H]":
        return precursor_mz -1
    elif adduct=="[M]+" or adduct=="[M]-":
        return precursor_mz
    else:
        return precursor_mz

In [ ]:
def get_trans_vec(x, dim=100):
    intensity_list = np.sqrt(x['intensities'])
    intensity_list = intensity_list/np.max(intensity_list)
    mzlist = x['mz']
    n_peaks = len(intensity_list)
    descend_order = intensity_list.argsort()[::-1]
    mzlist = mzlist[descend_order]
    intensity_list = intensity_list[descend_order]
    if n_peaks>dim:
        return np.stack([mzlist[:dim],intensity_list[:dim]]) # keep the first n peaks according to their intensity value
    else: # we pad it with zero if the number of peaks less than dim 
        intensity_list = np.pad(intensity_list, (0, dim-n_peaks), 'constant', constant_values=0)
        mzlist = np.pad(mzlist, (0, dim-n_peaks), 'constant', constant_values=0)
        return np.stack([mzlist,intensity_list])

def mgf2df(mgf_path,bin=True):    
    with open(mgf_path,'rb') as f:
        mgf = pickle.load(f) 
    data_collect = []
    for i in tqdm(mgf):
        if i:
            temp = {}
            temp['mz'] = i.mz
            temp['intensities']=i.intensities
            temp['inchikey'] = i.metadata.get("inchikey","-1")
            temp['precursor_mz'] = i.metadata.get("precursor_mz",0)
            temp['inchikey14'] = i.get("inchikey14",'0')
            temp['source_instrument'] = i.get('source_instrument','N/A-N/A')
            temp['adduct'] = i.get('adduct','0')
            temp['charge'] = i.get('charge')
            temp['smile'] = i.get('smiles')
            temp['precursor_mz_new'] = true_M(i.metadata.get("precursor_mz",0), i.get('adduct','0'))
            data_collect.append(temp)

    data = pd.DataFrame(data_collect)
    data["vec"] = data.apply(get_trans_vec, axis=1)
    data['m_id'] = data['inchikey14']
        
    return data

In [ ]:
def load_data(datapath, flag, bin):
    data = mgf2df(datapath, bin=bin)
    data["flag"] = flag
   
    data["precursor_mz"] = data["precursor_mz"]/1000
    data["precursor_mz_new"] = data["precursor_mz_new"]/1000
    data["charge"] = data['charge'].astype(int)

    cat_columns = ['adduct','source_instrument'] 
    for co in cat_columns:
        lb = LabelEncoder()
        lb.classes_ = np.load(co+"_classes.npy", allow_pickle=True)
        data[co] = lb.transform(data[co])

    data['index'] = list(range(data.shape[0]))
    return data

In [ ]:
def get_predict_results(data, model):
    predict_array = [np.stack(data['vec'].values),data['source_instrument'].values,data['charge'].values,np.reshape(data[PRECURSOR_MZ].values,(-1,1)),data['adduct'].values]
    train_result = []
    batch_size = 32
    for i in tqdm(range(0, data.shape[0],batch_size)):
        f = predict_array[0][i:i+batch_size] 
        t = predict_array[1][i:i+batch_size] 
        k = predict_array[2][i:i+batch_size] 
        l = predict_array[3][i:i+batch_size] 
        a = predict_array[4][i:i+batch_size] 
        m = [f,t,k,l,a]

        train_result.append(model(m).cpu().detach().numpy())
    predict_array_hidden = np.vstack(train_result)
    predict_array_hidden_norm = predict_array_hidden / np.linalg.norm(predict_array_hidden,axis=1).reshape(predict_array_hidden.shape[0],1)
    print("the data size ：", data.shape[0])
    return predict_array[0],predict_array_hidden,predict_array_hidden_norm

# load data

In [ ]:
data_train1 = load_data(abspath+ "data/" + "train_1.pickle",'train_1',False)
data_train2 = load_data(abspath+ "data/" + "train_2.pickle",'train_2',False)
data_test1 = load_data(abspath+ "data/" + "test_1.pickle",'test_1',False)
data_test2 = load_data(abspath+ "data/" + "test_2.pickle",'test_2',False)

In [ ]:
data_combine_all = pd.concat([data_train1,data_train2,data_test1,data_test2])

In [ ]:
# the data_index is a dict, and the key is inchikey, and the value is mass_spectrum instrument charge adduct, etc.
data_index = defaultdict(list)
for i in tqdm(range(data_train1.shape[0])):
    data_index[data_train1.iloc[i]['m_id']].append([data_train1.iloc[i]['vec'],data_train1.iloc[i]['source_instrument'],data_train1.iloc[i]['charge'],data_train1.iloc[i][PRECURSOR_MZ],data_train1.iloc[i]['adduct']])

# split the train and test
inchikey14_unique = data_train1["m_id"].unique()
inchikey14 = inchikey14_unique

l = len(inchikey14)
keyvalindex = np.random.choice(np.arange(l),int(val_ratio*l),replace=False) 
keyval = inchikey14[keyvalindex]
keytrain = np.delete(inchikey14,keyvalindex)

In [ ]:
# set the data used for train
dtype = torch.float32
score_index = score_df.index.values
score_dict = dict(zip(score_index,np.arange(len(score_index))))
train_data = ConTranDataGen(data_index, keytrain, score_dict,nround=10,nviews=2,augment=False)
val_data = ConTranDataGen(data_index, keyval,score_dict)
model = SinSiameseModel(100, project_size=project_size).to(dtype).to("cuda")
dataloader_train = DataLoader(train_data,256,shuffle=True)
dataloader_val = DataLoader(val_data,256)

# model training and save

In [ ]:
if train_flag:
    optimizer = torch.optim.Adam(model.parameters(),lr=5e-5,weight_decay=0.001)
    scheduler = torch.optim.lr_scheduler.StepLR(optimizer, 100, 0.7)

    trainer = TrainerCon(model, 
                            [dataloader_train,dataloader_val], 
                            score_df,
                            optimizer,epochs, 
                            scheduler,
                            dtype,
                            pth = pth,
                            model_name = model_name,
                            sim_ratio = sim_loss,
                            temperature = temperature)
    trainer.train_and_evaluate()

# load model

In [ ]:
best_model_results = model_results + "/models/best.pkl"

In [ ]:
device = 'cuda'
dtype = torch.float32
model = SinSiameseModel(100,project_size=project_size).to(device).to(dtype)
model.load_state_dict(torch.load(best_model_results))
model.eval()

# predict

In [ ]:
data_all_array, data_all_array_hidden,data_all_array_hidden_norm = get_predict_results(data_combine_all, model)